# The dynamic Malthusian macro model and a few extensions

## 0. <a id='toc0_'></a>[Preamble](#toc0_)

This model analysis project considers a simple dynamic Malthusian model. For the basic model setup we rely on the lecture note of Carl-Johan Dalgaard (2014), which provides a description on the simple dynamic Malthusian model based on Ashraf and Galor (2011). 

**Imports and set magics:**



In [13]:
# importing relevant modules 
import numpy as np
from scipy import optimize
import math
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# import pyfile with model solver
from modelproject import MalthusModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. <a id='toc1_'></a>[Model description](#toc0_)

**Time:** Discrete and indexed by $t\in\{0,1,\dots\}$.

**Production:** Production is described by a Cobb-Douglas production function that assumes that labor i subject to diminishing returns in production.

$$Y_t = (AX)^\alpha L_t^{1-\alpha}, \ \ \ 0<\alpha<1\tag{1}$$

where 
* $t$ denotes time 
* $Y_t$ is output
* $A$ is the constant level of technology
* $L_t$ is labor input
* $X$ is a fixed production factor (land)

For now we ignore that A could be growing: technological change is viewed merely as a series og discrete shocks.

Note that we can then write output per worker as 

$$y_t = (AX/L_t)^\alpha\tag{2}$$

The second key assumption is that the birth rate (i.e. births per capita) rises with per capita income. We can think of $\eta$ as being bounded below one, such that each parent in the ecnonomy uses a fixed fraction, $\eta$ of their income to rear children. Implicitly $\eta$ captures preferences for family size.

$$n_t=\eta y_t, \ \ \ 0<\eta<1\tag{3}$$

We assume no unemployment and the labor force evolves accoring to the transition equation

$$L_{t+1} = n_t L_t + (1-\mu) L_t, \ \ L_0 \ \text{given} \tag{4}$$  

Assuming people work until they die, we can interpret $\mu$ as reflecting mortality.

**Microfoundations of fertility**:

For simplicity assume that all agents receive the average income $y_t$. The budget constraint is then

$$c_t + \delta n_t = y_t(1-\tau) \tag{5}$$ 

where $c_t$ is consumpion an $\lambda$ is the relative price of children.

The utility function is 

$$u_t =\beta \mathrm{log}(c_t) + (1-\beta) \mathrm{log}(n_t) \tag{6}$$

By substitution of the budget constraint, the agents solves the following problem in when choosing between consumption of goods and expenses children.

$$\max_{n} u_t = \beta[y_t (1-\tau) -\delta n_t] + (1-\beta) \mathrm{log} (n_t) \tag{7}$$

## 2. <a id='toc2_'></a>[Solve household problem](#toc0_)

We can start by solving the model numerically for given income, $y=100$:

In [65]:
model = MalthusModelClass() # call class
model.solve() # solving model
print(f"Optimal consumption: {model.c:.2f}")
print(f"Optimal number of off spring: {model.n:.2f}")

Optimal consumption: 49.99
Optimal number of off spring: 16.67


We can also solve the model analytically the household problem analytically

In [70]:
# a. variables
u = sm.symbols('u')
y = sm.symbols('y')
c = sm.symbols('c')
n = sm.symbols('n')

# b. parameters
beta = sm.symbols('beta')
delta = sm.symbols('delta')
tau = sm.symbols('tau')

# c. utility function
u = beta*sm.ln(y*(1-tau)-delta*n)+(1-beta)*sm.log(n)
display(u)


beta*log(-delta*n + y*(1 - tau)) + (1 - beta)*log(n)

In [69]:
# d. differentiate wrt. to n 
du_dn = u.diff(n)
display(du_dn)

-beta*delta/(-delta*n + y*(1 - tau)) + (1 - beta)/n

In [76]:
# e. 
sol = sm.solve(sm.Eq(du_dn,0), n)
sol[0]


y*(beta*tau - beta - tau + 1)/delta

In the following parts we will do derivations using sympy but state the equilibrium relations.

## 3. <a id='toc2_'></a>[Simulating the model](#toc0_)